# D3M Tutorial

In this tutorial, we will use D3M to monitor deteriorating shifts in the UCI Heart Disease dataset. The preprocessed dataset is available in ``data/uci_data/``.

In [1]:
import os
os.chdir('../')
import torch
import numpy as np

SEED = 57
np.random.seed(SEED)
torch.random.manual_seed(SEED)

### Import data

We begin with importing data:

In [2]:
from data.uci_data import UCIDataset
data_dict = torch.load('data/uci_data/uci_heart_torch.pt')
uci_dict = {}
for k, data in data_dict.items():
    data = list(zip(*data))
    X, y = torch.stack(data[0]), torch.tensor(data[1], dtype=torch.int)
    
    # normalize
    min_ = torch.min(X, dim=0).values
    max_ = torch.max(X, dim=0).values
    X = (X - min_) / (max_ - min_)
    uci_dict[k] = UCIDataset(X, y)

We will use ``train`` to train the base model, ``valid`` to validate the base model. Then, ``valid`` will be used to train the distribution of i.i.d. disagreement rates Phi.

We will monitor on both ``train`` and ``iid_test`` in order to validate that our monitor is well-calibrated, i.e. it resists flagging in-distribution unseen samples. 

Finally, we monitor ``ood_test`` to assert that our monitor detects deteriorating changes from the dataset.

In [3]:
dataset_dict = {}
dataset_dict['train'] = uci_dict['train']
dataset_dict['valid'] = uci_dict['val']
dataset_dict['d3m_train'] = uci_dict['val']
dataset_dict['d3m_id'] = uci_dict['iid_test']
dataset_dict['d3m_ood'] = uci_dict['ood_test']

### Import D3M components

D3M involves two primary components: 
    
- the base model
- the monitor

The base model will depend on the data modality. For tabular data, we work with ``MLPModel``.  For images, we work with ``ConvModel``.
The monitor takes in a base model, training and validation datasets, and a configuration file. We parse these using ``hydra-core``.

In [4]:
import hydra
from omegaconf import DictConfig
from d3m import MLPModel, D3MBayesianMonitor
from experiments.utils import get_configs

In [5]:
hydra.initialize(config_path='../experiments/configs', version_base='1.2')
args = hydra.compose(config_name="uci_best")

In [6]:
model_config, train_config = get_configs(args)

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
base_model = MLPModel(model_config, train_size=len(dataset_dict['train']))
monitor = D3MBayesianMonitor(
        model=base_model,
        trainset=dataset_dict['train'],
        valset=dataset_dict['valid'],
        train_cfg=train_config,
        device=device,
    )

### Base Classifier Training

We are now ready to train the model per our configurations. Simply run ``monitor.train_model``. 

In [8]:
ood_testloader = torch.utils.data.DataLoader(dataset_dict['d3m_ood'],
                                                     batch_size=train_config.batch_size,
                                                     shuffle=False,
                                                     num_workers=train_config.num_workers,
                                                     pin_memory=train_config.pin_memory)
output_metrics = monitor.train_model(tqdm_enabled=True, testloader=ood_testloader)

  0%|          | 0/50 [00:00<?, ?it/s]

Epoch:  0, train loss: 19.4199	train accuracy: 0.5706


Epoch:  0, val loss: 0.6563	val accuracy: 0.6317


  2%|▏         | 1/50 [00:00<00:37,  1.30it/s]

Epoch:  0, ood test loss: 0.7521	ood test accuracy: 0.3264


...

Epoch: 48, ood test loss: 0.6566	ood test accuracy: 0.6632


Epoch: 49, train loss: 13.9764	train accuracy: 0.7982


Epoch: 49, val loss: 0.4781	val accuracy: 0.7567


100%|██████████| 50/50 [00:20<00:00,  2.44it/s]

Epoch: 49, ood test loss: 0.6554	ood test accuracy: 0.6580


### Training of the maximum i.i.d. disagreement rate distribution

We now have a base Bayesian model fitted to the training data. We now seek the disagreement rate with respect to our base classifier of models that:

- agree with our base classifier on the training data
- disagree with our base classifier on unseen i.i.d. data

In order to achieve this, we batch sample from our belief over the decision surface, and repeatedly select the decision surface with the strongest disagreement rate. We collect these disagreement rates into model.Phi.


In [9]:
monitor.pretrain_disagreement_distribution(dataset=dataset_dict['d3m_train'],
                                           n_post_samples=args.d3m.n_post_samples,
                                           data_sample_size=args.d3m.data_sample_size,
                                           Phi_size=args.d3m.Phi_size, 
                                           temperature=args.d3m.temp,
                                           )

100%|██████████| 1000/1000 [00:03<00:00, 274.17it/s]


### Compute FPRs and TPRs

We are essentially done. Our monitor has the essential components:

- a trained base classifier on i.i.d. data
- a trained distribution of i.i.d. disagreement rates

This base classifier is now ready to be deployed on any deployment data, as long as we monitor periodically by running either ``monitor.d3m_test`` or ``monitor.repeat_tests`` (for repeated testing, useful to compute statistics) on future data.

In [10]:
stats = {}
dis_rates = {}

for k,dataset in {
    'd3m_train': dataset_dict['d3m_train'],
    'd3m_id': dataset_dict['d3m_id'],
    'd3m_ood': dataset_dict['d3m_ood']
}.items():
    rate, max_dis_rates = monitor.repeat_tests(n_repeats=args.d3m.n_repeats,
                                    dataset=dataset, 
                                    n_post_samples=args.d3m.n_post_samples,
                                    data_sample_size=args.d3m.data_sample_size,
                                    temperature=args.d3m.temp
                                    )
    print(f"{k}: {rate}")
    stats[k] = rate
    dis_rates[k] = (np.mean(max_dis_rates), np.std(max_dis_rates))


100%|██████████| 100/100 [00:00<00:00, 272.35it/s]


d3m_train: 0.02


100%|██████████| 100/100 [00:00<00:00, 275.26it/s]


d3m_id: 0.03


100%|██████████| 100/100 [00:00<00:00, 274.74it/s]

d3m_ood: 1.0


In the above, we notice that the TPR for OOD samples is 1.0, i.e. our bayesian D3M monitor is able to correctly identify deteriorating changes in the data distribution.

We further notice that for a held-out in-distribution sample, the model does not identify the sample as out-of-distribution. Indeed, the base classifier achieves similar performance on this held-out set than on the validation set, proving that bayesian D3M incurs low false positive rates for in-distribution samples.